# MUMBAI

In [12]:
# import os,sys;sys.path.append(os.path.dirname(os.path.realpath('mail')))
# from mail.send_mail import sendMail
import pygsheets #Importing python in google sheets
import pandas as pd #importing pandas
from pandas import DataFrame
import datetime as dt
from datetime import datetime, timedelta,date
import numpy as np
import datetime

import warnings
warnings.filterwarnings('ignore')

# try:
#     gc = pygsheets.authorize(service_file='/home/karan/config/car-master-sheet.json')
# gc = pygsheets.authorize(service_file='car-master-sheet.json')
# gc = pygsheets.authorize(service_file='/Users/karan_19981/Desktop/Work/Everest_Fleet/Database_Sheets/client_secret.json')


# gc = pygsheets.authorize(service_file='/home/ubuntu/scripts/client_secret.json')

gc = pygsheets.authorize(service_file='car-master-sheet.json')


today = date.today() + timedelta(days = 1)

allotment_wkbk = gc.open("Allotment Status Report")


allotment_history_sheet = allotment_wkbk.worksheet_by_title("New Allotment History")

allotment_history_data = pd.DataFrame(allotment_history_sheet.get_all_records())

allotment_history_data = allotment_history_data[['Car + Count','Car Number','Allotment Date','ETM','Return Date','Allotment Fastag']]

allotment_history_data['Allotment Fastag']=allotment_history_data['Allotment Fastag'].str.replace(',','')

allotment_history_data['Allotment Fastag']=pd.to_numeric(allotment_history_data['Allotment Fastag'], errors = 'coerce')

allotment_history_data['Return Date'] = allotment_history_data['Return Date'].replace(r'^\s*$', today, regex=True)

allotment_history_data = allotment_history_data.replace(r'^\s*$', np.nan, regex=True)

allotment_history_data = allotment_history_data[allotment_history_data['ETM'].str.contains('ET',na=False)]

allotment_history_data['Allotment Date'] = pd.to_datetime(allotment_history_data['Allotment Date'], dayfirst = True)

allotment_history_data['Return Date'] = pd.to_datetime(allotment_history_data['Return Date'], dayfirst = True)


allotment_history_data['Allotment Date Dummy'] = allotment_history_data['Allotment Date'].dt.date

allotment_history_data['Return Date Dummy'] = allotment_history_data['Return Date'].dt.date

adjustment_given_allotment_history = allotment_history_data[['Car Number','Return Date Dummy','Return Date','ETM']]

adjustment_taken_allotment_history = allotment_history_data[['Car Number','ETM','Allotment Date Dummy','Allotment Date','Allotment Fastag','Return Date']]

adjustment_taken_allotment_history['Allotment Fastag']=adjustment_taken_allotment_history['Allotment Fastag'].replace(',','')

audit_data = allotment_wkbk.worksheet_by_title("Audit")

allotment_audit_data = pd.DataFrame(audit_data.get_all_records())

allotment_audit_data = allotment_audit_data[['Car Number','Fastag Balance','Timestamp']]

allotment_audit_data = allotment_audit_data.replace(r'^\s*$', np.nan, regex=True)

allotment_audit_data = allotment_audit_data[allotment_audit_data['Fastag Balance'].notna()]

allotment_audit_data['Timestamp'] = pd.to_datetime(allotment_audit_data['Timestamp'], dayfirst = True)


allotment_audit_data['start_date'] = pd.Timestamp('08-07-2022')#changes

allotment_audit_data = allotment_audit_data.loc[(allotment_audit_data['Timestamp'] > allotment_audit_data['start_date'])]

allotment_audit_data['Fastag Balance']=allotment_audit_data['Fastag Balance'].replace(',','')

allotment_audit_data = allotment_audit_data.loc[~(allotment_audit_data['Fastag Balance'] == 0)]

allotment_audit_data['Return Date'] = allotment_audit_data['Timestamp']

allotment_audit_data.drop(['start_date'],axis = 1, inplace = True)#changes

allotment_audit_data



adjustment_given = pd.merge(allotment_audit_data,adjustment_given_allotment_history,on=['Return Date','Car Number'],how='inner')
adjustment_given.rename(columns={"Return Date Dummy":"Bill Date"},inplace = True)


adjustment_given['start_date'] = pd.Timestamp('08-07-2022')#changes
adjustment_given = adjustment_given.loc[(adjustment_given['Timestamp'] > adjustment_given['start_date'])]
adjustment_given.drop(['start_date'],axis = 1, inplace = True)
adjustment_given['Adjustment Reason'] = 'Audit Fastag' 
adjustment_given['Adjustment Comment'] = 'Audit Fastag' 

adjustment_given['Adjustment Amount'] = adjustment_given['Fastag Balance']
adjustment_given.drop(['Fastag Balance'],axis = 1, inplace = True)

adjustment_given

allotment_audit_data.drop(['Timestamp'],axis = 1, inplace = True)

adjustment_taken = adjustment_taken_allotment_history.copy()


adjustment_taken['Adjustment Amount New'] = adjustment_taken['Allotment Fastag'] * -1
adjustment_taken['Adjustment Reason'] = 'Allotment Fastag' 
adjustment_taken['Adjustment Comment'] = 'Allotment Fastag' 
adjustment_taken['start_date'] = pd.Timestamp('08-07-2022')#changes
adjustment_taken = adjustment_taken.loc[(adjustment_taken['Allotment Date'] > adjustment_taken['start_date'])]
adjustment_taken.drop(['start_date'],axis = 1, inplace = True)

adjustment_taken


adjustment_taken.drop(['Allotment Fastag'],axis = 1, inplace = True)


adjustment_taken.rename(columns={"Allotment Date Dummy":"Bill Date","Allotment Date":"Timestamp","Adjustment Amount New":"Adjustment Amount"},inplace = True)




print(adjustment_given.head())
print(adjustment_taken.head())


adjustment_final = pd.concat([adjustment_taken, adjustment_given], ignore_index=True)

adjustment_final['Name of DM'] = 'Allotment'
adjustment_final['Pilot Name'] = ''

adjustment_final.sort_values(by=['Timestamp'], ascending=True, inplace=True)


adjustment_final = adjustment_final.dropna(subset=['Adjustment Amount'])
 
adjustment_final = adjustment_final.reset_index(drop=True)
 
adjustment_final


# adjustment_gs = gc.open("New-Adjustment Sheet")

# adjustment_sheet = adjustment_gs.worksheet_by_title("New Allotment Fastag")

# adjustment_sheet.set_dataframe(adjustment_final, 'A1')


# except Exception:
#     sendMail([
#     'abusalameverestfleet@gmail.com',
#     'avinash.everestfleet@gmail.com',
#     'rutvijaevrestfleet@gmail.com',
#     'sameer.everestfleet@gmail.com',
#     'Karan.saraogi@everestfleet.com',
#     'vijugode@gmail.com'
#     ])

   Car Number           Timestamp         Return Date   Bill Date       ETM  \
0  MH01DR5875 2022-08-07 08:54:41 2022-08-07 08:54:41  2022-08-07  ETM23996   
1  MH01DR4332 2022-08-08 10:33:41 2022-08-08 10:33:41  2022-08-08  ETM24027   
2  MH01DR8912 2022-08-08 13:14:58 2022-08-08 13:14:58  2022-08-08  ETM22465   
3  MH01DR5469 2022-08-08 13:57:10 2022-08-08 13:57:10  2022-08-08  ETM23791   
4  MH01DR8096 2022-08-08 15:06:34 2022-08-08 15:06:34  2022-08-08  ETM22307   

  Adjustment Reason Adjustment Comment Adjustment Amount  
0      Audit Fastag       Audit Fastag             -42.5  
1      Audit Fastag       Audit Fastag                60  
2      Audit Fastag       Audit Fastag               100  
3      Audit Fastag       Audit Fastag                 1  
4      Audit Fastag       Audit Fastag               100  
       Car Number       ETM   Bill Date           Timestamp  \
11782  MH01DR2403  ETM23147  2022-08-10 2022-08-10 16:43:02   
11870  MH01CV8645  ETM11240  2022-08-11 2022-

,Car Number,ETM,Bill Date,Timestamp,Return Date,Adjustment Amount,Adjustment Reason,Adjustment Comment,Name of DM,Pilot Name
0,MH01DR5875,ETM23996,2022-08-07,2022-08-07 08:54:41,2022-08-07 08:54:41,-42.5,Audit Fastag,Audit Fastag,Allotment,
1,MH01DR4332,ETM24027,2022-08-08,2022-08-08 10:33:41,2022-08-08 10:33:41,60,Audit Fastag,Audit Fastag,Allotment,
2,MH01DR8912,ETM22465,2022-08-08,2022-08-08 13:14:58,2022-08-08 13:14:58,100,Audit Fastag,Audit Fastag,Allotment,
3,MH01DR5469,ETM23791,2022-08-08,2022-08-08 13:57:10,2022-08-08 13:57:10,1,Audit Fastag,Audit Fastag,Allotment,
4,MH01DR8096,ETM22307,2022-08-08,2022-08-08 15:06:34,2022-08-08 15:06:34,100,Audit Fastag,Audit Fastag,Allotment,
...,...,...,...,...,...,...,...,...,...,...
61,MH01CV6958,ETM24796,2022-08-16,2022-08-16 16:11:51,2022-08-16 16:11:51,1,Audit Fastag,Audit Fastag,Allotment,
62,MH01DR5779,ETM03213,2022-08-16,2022-08-16 16:43:46,2022-08-16 16:43:46,77,Audit Fastag,Audit Fastag,Allotment,
63,MH01DR7256,ETM06141,2022-08-16,2022-08-16 17:20:05,2022-08-16 17:20:05,1,Audit Fastag,Audit Fastag,Allotment,
64,MH01DR8656,ETM11571,2022-08-16,2022-08-16 18:12:22,2022-08-16 18:12:22,1,Audit Fastag,Audit Fastag,Allotment,


In [13]:
adjustment_gs = gc.open("New-Adjustment Sheet")

adjustment_sheet = adjustment_gs.worksheet_by_title("New Allotment Fastag-test")

adjustment_sheet.set_dataframe(adjustment_final, 'A1')

In [4]:
allotment_wkbk = gc.open("Allotment Status Report")


allotment_history_sheet = allotment_wkbk.worksheet_by_title("New Allotment History")

allotment_history_data = pd.DataFrame(allotment_history_sheet.get_all_records())

allotment_history_data = allotment_history_data[['Car + Count','Car Number','Allotment Date','ETM','Return Date','Allotment Fastag']]

allotment_history_data['Allotment Fastag']=allotment_history_data['Allotment Fastag'].str.replace(',','')

allotment_history_data['Allotment Fastag']=pd.to_numeric(allotment_history_data['Allotment Fastag'], errors = 'coerce')

allotment_history_data['Return Date'] = allotment_history_data['Return Date'].replace(r'^\s*$', today, regex=True)

allotment_history_data = allotment_history_data.replace(r'^\s*$', np.nan, regex=True)

allotment_history_data = allotment_history_data[allotment_history_data['ETM'].str.contains('ET',na=False)]

allotment_history_data['Allotment Date'] = pd.to_datetime(allotment_history_data['Allotment Date'], dayfirst = True)

allotment_history_data['Return Date'] = pd.to_datetime(allotment_history_data['Return Date'], dayfirst = True)

adjustment_gs = gc.open("New-Adjustment Sheet")

adjustment_sheet = adjustment_gs.worksheet_by_title("allotment history")

adjustment_sheet.set_dataframe(allotment_history_data, 'A1')

In [5]:
allotment_history_data.dtypes

Car + Count                 object
Car Number                  object
Allotment Date      datetime64[ns]
ETM                         object
Return Date         datetime64[ns]
Allotment Fastag           float64
dtype: object

In [30]:
allotment_history_data.dtypes

Car + Count                 object
Car Number                  object
Allotment Date      datetime64[ns]
ETM                         object
Return Date         datetime64[ns]
Allotment Fastag            object
dtype: object

In [ ]:
# import os,sys;sys.path.append(os.path.dirname(os.path.realpath('mail')))
# from mail.send_mail import sendMail
import pygsheets #Importing python in google sheets
import pandas as pd #importing pandas
from pandas import DataFrame
import datetime as dt
from datetime import datetime, timedelta,date
import numpy as np
import datetime

import warnings
warnings.filterwarnings('ignore')

# try:
#     gc = pygsheets.authorize(service_file='/home/karan/config/car-master-sheet.json')
# gc = pygsheets.authorize(service_file='car-master-sheet.json')
# gc = pygsheets.authorize(service_file='/Users/karan_19981/Desktop/Work/Everest_Fleet/Database_Sheets/client_secret.json')


# gc = pygsheets.authorize(service_file='/home/ubuntu/scripts/client_secret.json')

gc = pygsheets.authorize(service_file='car-master-sheet.json')


today = date.today() + timedelta(days = 1)

allotment_wkbk = gc.open("Allotment Status Report")


allotment_history_sheet = allotment_wkbk.worksheet_by_title("New Allotment History")

allotment_history_data = pd.DataFrame(allotment_history_sheet.get_all_records())

allotment_history_data = allotment_history_data[['Car + Count','Car Number','Allotment Date','ETM','Return Date','Allotment Fastag']]

allotment_history_data['Return Date'] = allotment_history_data['Return Date'].replace(r'^\s*$', today, regex=True)

allotment_history_data = allotment_history_data.replace(r'^\s*$', np.nan, regex=True)

allotment_history_data = allotment_history_data[allotment_history_data['ETM'].str.contains('ET',na=False)]

allotment_history_data['Allotment Date'] = pd.to_datetime(allotment_history_data['Allotment Date'], dayfirst = True)

allotment_history_data['Return Date'] = pd.to_datetime(allotment_history_data['Return Date'], dayfirst = True)


allotment_history_data['Allotment Date Dummy'] = allotment_history_data['Allotment Date'].dt.date

allotment_history_data['Return Date Dummy'] = allotment_history_data['Return Date'].dt.date

adjustment_given_allotment_history = allotment_history_data[['Car Number','Return Date Dummy','Return Date','ETM']]

adjustment_taken_allotment_history = allotment_history_data[['Car Number','ETM','Allotment Date Dummy','Allotment Date','Allotment Fastag']]

audit_data = allotment_wkbk.worksheet_by_title("Audit")

allotment_audit_data = pd.DataFrame(audit_data.get_all_records())

allotment_audit_data = allotment_audit_data[['Car Number','Fastag Balance','Timestamp']]

allotment_audit_data = allotment_audit_data.replace(r'^\s*$', np.nan, regex=True)

allotment_audit_data = allotment_audit_data[allotment_audit_data['Fastag Balance'].notna()]

allotment_audit_data['Timestamp'] = pd.to_datetime(allotment_audit_data['Timestamp'], dayfirst = True)


allotment_audit_data['start_date'] = pd.Timestamp('08-07-2022')#changes

allotment_audit_data = allotment_audit_data.loc[(allotment_audit_data['Timestamp'] > allotment_audit_data['start_date'])]

allotment_audit_data = allotment_audit_data.loc[~(allotment_audit_data['Fastag Balance'] == 0)]

allotment_audit_data['Return Date'] = allotment_audit_data['Timestamp']

allotment_audit_data.drop(['start_date'],axis = 1, inplace = True)#changes



adjustment_given = pd.merge(allotment_audit_data,adjustment_given_allotment_history,on=['Return Date','Car Number'],how='inner')
adjustment_given.rename(columns={"Return Date Dummy":"Bill Date"},inplace = True)

# adjustment_given['start_date'] = pd.Timestamp('08-02-2022')
adjustment_given['start_date'] = pd.Timestamp('08-07-2022')#changes
adjustment_given = adjustment_given.loc[(adjustment_given['Timestamp'] > adjustment_given['start_date'])]
adjustment_given.drop(['start_date'],axis = 1, inplace = True)
adjustment_given['Adjustment Reason'] = 'Audit Fastag' 
adjustment_given['Adjustment Comment'] = 'Audit Fastag' 
adjustment_given['Adjustment Amount'] = adjustment_given['Fastag Balance']
adjustment_given.drop(['Fastag Balance'],axis = 1, inplace = True)



allotment_audit_data.drop(['Timestamp'],axis = 1, inplace = True)

adjustment_taken = adjustment_taken_allotment_history.copy()
adjustment_taken['Adjustment Amount New'] = adjustment_taken['Allotment Fastag'] * -1
adjustment_taken['Adjustment Reason'] = 'Allotment Fastag' 
adjustment_taken['Adjustment Comment'] = 'Allotment Fastag' 
adjustment_taken['start_date'] = pd.Timestamp('08-07-2022')#changes
adjustment_taken = adjustment_taken.loc[(adjustment_taken['Allotment Date'] > adjustment_taken['start_date'])]
adjustment_taken.drop(['start_date'],axis = 1, inplace = True)



adjustment_taken

adjustment_taken.drop(['Allotment Fastag'],axis = 1, inplace = True)

# adjustment_taken.drop(['Adjustment Amount'],axis = 1, inplace = True)


adjustment_taken.rename(columns={"Allotment Date Dummy":"Bill Date","Allotment Date":"Timestamp","Adjustment Amount New":"Adjustment Amount"},inplace = True)




print(adjustment_given.head())
print(adjustment_taken.head())


adjustment_final = pd.concat([adjustment_taken, adjustment_given], ignore_index=True)


adjustment_final['Name of DM'] = 'Allotment'
adjustment_final['Pilot Name'] = ''

adjustment_final.sort_values(by=['Timestamp'], ascending=True, inplace=True)

adjustment_final


# adjustment_gs = gc.open("New-Adjustment Sheet")

# adjustment_sheet = adjustment_gs.worksheet_by_title("New Allotment Fastag")

# adjustment_sheet.set_dataframe(adjustment_final, 'A1')


# except Exception:
#     sendMail([
#     'abusalameverestfleet@gmail.com',
#     'avinash.everestfleet@gmail.com',
#     'rutvijaevrestfleet@gmail.com',
#     'sameer.everestfleet@gmail.com',
#     'Karan.saraogi@everestfleet.com',
#     'vijugode@gmail.com'
#     ])